# 10K Analysis - Sub Question Query Engine

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [1]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

/Users/suo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


configure LLM

In [2]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

load data 

In [3]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

build indices

In [4]:
lyft_index = GPTVectorStoreIndex.from_documents(lyft_docs)

In [5]:
uber_index = GPTVectorStoreIndex.from_documents(uber_docs)

configure query engine

In [6]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [7]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

build sub-question query engine

In [8]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [12]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

In [ ]:
response = s_engine.query('Compare and contrast the customer segments and geographies that grew the fastest')

In [ ]:
print(response)